installations

In [1]:
!pip install skfeature-chappers

imports:

Note : ifs_to_change - is the algorithm (ifs) after the improment i made

In [2]:
from skfeature.function.similarity_based.reliefF import reliefF
from skfeature.function.information_theoretical_based.MRMR import mrmr
import scipy.io
import pandas as pd
import numpy as np
import time
from sklearn.svm import SVC, SVR
from sklearn.feature_selection import RFE, SelectFdr
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import PowerTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import matthews_corrcoef, roc_auc_score, accuracy_score, precision_recall_curve, roc_curve, auc
from sklearn.model_selection import KFold, LeaveOneOut, LeavePOut
from sklearn.preprocessing import LabelEncoder, label_binarize
import os
import ifs_to_change as ifs
from tqdm import tqdm

The func_score is the call tp the function after improvment:

In [3]:
def func_score(x,y):
    inf = ifs.InfFS()
    alpha = 0.8
    supervision = 1
    verbose = 0
    [RANKED, WEIGHT] = inf.infFS(x, y, alpha, supervision, verbose)
    return RANKED, WEIGHT

Collecting the datasets (option for .csv files and for .mat files)

the datasets are saved in an extarnmel dir named "MAT" (under the same dir as the nootebook)

In [4]:
data_files = [("./MAT/"+f) for f in os.listdir('./MAT/') if '.csv' in f]

## for mat files: 
## data_files = [("./MAT/"+f) for f in os.listdir('./MAT/') if '.mat' in f]


Required lists for the main loop:


classifiers : all the classifiers we use

top_k_features: list of all required k values

featire_selection_func_list : contains only the improved func

In [5]:
classifiers = [RandomForestClassifier(), LogisticRegression(), KNeighborsClassifier(), 
               GaussianNB(), SVC(kernel='linear',probability=True)]
top_k_features_to_take = [1,2,3,4,5,10,15,20,25,30,50,100] 
feature_selection_func_list = [func_score]

display the datasets:

In [6]:
data_files

['./MAT/pone005.csv']

The organization data structure:

In [7]:
raw_data = {
    'Dataset Name' : [],
    'Number of samples' : [],
    'Original Number of features' : [],
    'Filtering Algorithm' : [],
    'Learning algorithm': [],
    'Number of features selected (K)': [],
    'CV Method' : [],
    'Fold': [],
    'accurecy_score_val' : [],
    'matthews_corrcoef val' : [],
    'roc_auc_score val' : [],
    'precision_recall_curve val' : [],
    'feature selection time': [],
    'fit time': [],
    'Inference time': [],
    'List of Selected Features Names': [],
    'Selected Features scores': []
}

the reading function, and process dataframe

In [8]:
def read_file(path_to_data):
    """
    reading a .csv file
    getting the feature names
    
    * I'm dyslexic and by mistac I somethimes wrote target and sometimes tergat, hence the try-catch
    
    """
    df = pd.read_csv(path_to_data)
    feature_names = df.columns.to_list()
    try:
        feature_names.remove('tergat')
    except:
            try:
                feature_names.remove('target')
            except:
                return df, feature_names
    return df, feature_names


#def read_file(path_to_data):
#    """
#    read file for .mat file
#    """
#    data_dict = scipy.io.loadmat(path_to_data)
#    x = data_dict['X']
#    x = x.astype('float')
#    y = data_dict['Y']
#    le = LabelEncoder()
#    y = le.fit_transform(y)
#    feature_names = [f'feature_{i}' for i in range(x.shape[1])]
#    df = pd.DataFrame(data=x, columns=feature_names)
#    df['target'] = y
#    feature_names = df.columns.to_list()
#    feature_names.remove('target')
#    return df, feature_names

def process_dataframe(df, feature_names):
    """
    claculating missing vals by mean value and droping variables with variance zero
    """

    si = SimpleImputer(strategy='mean')
    df[feature_names] = si.fit_transform(df[feature_names].values)

    pt = PowerTransformer()
    df[feature_names] = pt.fit_transform(df[feature_names].values)
    
    column_vars = df.var()
    zero_variance_columns = list(column_vars[column_vars == 0].index)
    df.drop(columns=zero_variance_columns, inplace=True)


    return df

In [9]:
def get_micro_auc_roc(x, y, y_pred, n_class):
    is_multi_class = n_class > 2
    if is_multi_class:
        fpr = dict()
        tpr = dict()
        roc_auc = dict()
        y_one = label_binarize(y, classes=range(n_class)) 
        fpr["micro"], tpr["micro"], _ = roc_curve(y_one.ravel(), y_pred.ravel())
        roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])
        return roc_auc["micro"]
    else:
        return roc_auc_score(y, y_pred[:,1])

def get_precision_recall_auc(x, y, y_pred, n_class):
    is_multi_class = n_class > 2
    if is_multi_class:
        y_one = label_binarize(y, classes= range(n_class))
        # Compute micro-average ROC curve and ROC area
        precision, recall, _ = precision_recall_curve(y_one.ravel(), y_pred.ravel())
        return  auc(recall, precision)
    else:
        precision, recall, thresholds = precision_recall_curve(y, y_pred[:,1])
        return auc(recall, precision)

saving func

In [10]:
def save_result(df):
    df.to_csv("./res_part_c.csv")

The main loop:

In [11]:
from collections import defaultdict

for data_file_path in data_files[:1]:
    #giong throw the datasets
    file_name = data_file_path.split('/')[-1]
    print(file_name)
    df, feature_names = read_file(data_file_path)
    df = process_dataframe(df, feature_names)

    try:
        x, y = df[feature_names].values, df['tergat'].values
    except:
        x, y = df[feature_names].values, df['target'].values
    n_class = len(set(y))
    if df.shape[0] < 50:
        validation = LeavePOut(2)
        cv_name = 'Leave-pair-out'

    if 50 <= df.shape[0] <= 100:
        validation = LeaveOneOut()
        cv_name = 'Leave-one-out'

    if 100 < df.shape[0] <= 1000:
        validation = KFold(n_splits =10)
        cv_name = 'Kfold-10'

    if 1000 < df.shape[0]:
        validation = KFold(n_splits =5)
        cv_name = 'Kfold-5'

    probabilities_acc = defaultdict(list)
    real_labels = defaultdict(list)
    for fold_index,(train_index, test_index) in enumerate(validation.split(x)):
        #if so, no calcultion can be done as saud by professor Rockech
        if type(validation) is LeavePOut and (len(np.unique(y[test_index]))==1 or len(np.unique(y[train_index]))==1):
            continue
        
        x_train, x_test = x[train_index], x[test_index]
        y_train, y_test = y[train_index], y[test_index]
        for feature_selection_function in feature_selection_func_list:
            start_time_selected_feat = time.time()
            selected_features, selected_features_scores = feature_selection_function(x_train, y_train)
            end_time_selected_feat = time.time() - start_time_selected_feat
            for k in top_k_features_to_take:
                k_indexes = selected_features[:k]
                k_scores = selected_features_scores[:k]
                x_train_selected_features = x_train[:, k_indexes]
                x_test_selected_features = x_test[:, k_indexes]
                for model_class in tqdm(classifiers):
                    # calculations save the required info for the final excel
                    raw_data['Learning algorithm'].append(model_class.__class__.__name__)
                    model = model_class
                    start_fit_time = time.time()
                    model.fit(x_train_selected_features, y_train)
                    end_fit_time = time.time() - start_fit_time
                    raw_data['fit time'].append(end_fit_time)
                    y_predict = model.predict(x_test_selected_features) 
                    start_predict_time = time.time()
                    y_pred_proba = model.predict_proba(x_test_selected_features)
                    tot_prob_time = time.time() - start_predict_time
                    raw_data['Inference time'].append(tot_prob_time/x_test_selected_features.shape[0])
                    accurecy_score_val = accuracy_score(y_predict, y_test)
                    raw_data['accurecy_score_val'].append(accurecy_score_val)
                    matthews_corroef_val = matthews_corrcoef(y_predict, y_test)
                    raw_data['matthews_corrcoef val'].append(matthews_corroef_val)
                    if not (len(np.unique(y[test_index]))==1):
                        roc_auc_val = get_micro_auc_roc(x_test,y_test,y_pred_proba, n_class)
                        raw_data['roc_auc_score val'].append(roc_auc_val)
                        precision_val = get_precision_recall_auc(x_test,y_test,y_pred_proba, n_class)
                        raw_data['precision_recall_curve val'].append(precision_val)
                    else:
                        # in that case we will calaculate it latter for the remains, as said by Lior in the mail
                        raw_data['roc_auc_score val'].append(np.nan)
                        raw_data['precision_recall_curve val'].append(np.nan)
                        real_labels[(model_class.__class__.__name__, k, feature_selection_function.__name__)].append(y_test[0])
                        probabilities_acc[(model_class.__class__.__name__, k, feature_selection_function.__name__)].append(y_pred_proba)
                    raw_data['Filtering Algorithm'].append(feature_selection_function.__name__)
                    raw_data['feature selection time'].append(end_time_selected_feat)
                    raw_data['Number of features selected (K)'].append(k)
                    raw_data['Dataset Name'].append(file_name+'_New')
                    raw_data['Number of samples'].append(df.shape[0])
                    raw_data['Original Number of features'].append(df.shape[1]-1) #without the y's
                    raw_data['CV Method'].append(cv_name)
                    raw_data['Fold'].append(fold_index)
                    raw_data['Selected Features scores'].append(k_scores)
                    raw_data['List of Selected Features Names'].append(k_indexes)
    raw_data_df = pd.DataFrame.from_dict(raw_data)
    save_result(raw_data_df)

for model_name, k_val, func_name in list(probabilities_acc.keys()):
    y_real_lou = real_labels[(model_name, k_val, func_name)]
    y_pred_lou = np.concatenate(probabilities_acc[(model_name, k_val, func_name)], axis=0)
    try:
        lou_auc = get_micro_auc_roc(_, y_real_lou, y_pred_lou, n_class)
    except:
        lou_auc = np.nan
    try:
        lou_prec_recall_auc = get_precision_recall_auc(_, y_real_lou, y_pred_lou, n_class)
    except:
        lou_prec_recall_auc = np.nan
    part1 = ((raw_data_df['Learning algorithm'] == model_name) & (raw_data_df['Number of features selected (K)'] == k_val))
    part2 = part1 & (raw_data_df['Filtering Algorithm'] == func_name)
    index_to_change = raw_data_df[part2].index
    raw_data_df.loc[index_to_change, 'roc_auc_score val'] = lou_auc
    raw_data_df.loc[index_to_change, 'precision_recall_curve val'] = lou_prec_recall_auc
save_result(raw_data_df)

pone005.csv


100%|██████████| 5/5 [00:00<00:00, 28.52it/s]


100%|██████████| 5/5 [00:00<00:00, 40.31it/s]


100%|██████████| 5/5 [00:00<00:00, 31.38it/s]
